In [1]:
from vnstock import Vnstock
from vnstock import Listing
import talib as ta 
import mplfinance as mpf
import pandas as pd
import numpy as np


Hàm thực hiện mua bán dùng chỉ số CCI:
* Mua ngay khi gặp tín hiệu mua và bán ngay khi gặp tín hiệu bán
* Chỉ mua 1 lần đầu nếu gặp nhiều tín hiệu mua
* Lợi nhuận mỗi khi bán = giá đóng cửa khi bán - giá đóng cửa khi mua 

In [2]:
def test(symbol = 'AAA', start = '2018-01-01', end = '2025-01-01'):
    # print(f"CCI strategy for {symbol} from {start} to {end}")
    if symbol is not None:
        try:
            stock = Vnstock().stock(symbol)
            df = stock.quote.history(start = start, end = end, interval = '1D')
            

            df['time'] = pd.DatetimeIndex(df['time'])
            df.set_index('time', inplace=True)

            cci = ta.CCI(df['high'], df['low'], df['close'], timeperiod=14)

            profits = []
            in_position = False
            buy_price = 0

            for i in range(len(df)):
                if cci.iloc[i-1] < -100 and cci.iloc[i] > -100 and not in_position:
                    buy_price = df['close'].iloc[i]
                    in_position = True
                    # print(f"Buy at {buy_price} on {df.index[i]}")
                elif cci.iloc[i-1] > 100 and cci.iloc[i] < 100 and in_position:
                    sell_price = df['close'].iloc[i]
                    profits.append(sell_price - buy_price)
                    in_position = False
                    # print(f"Sell at {sell_price} on {df.index[i]}")

            return pd.Series(profits)
        
        except Exception as e:
            print(f"Error fetching data for {symbol}: {e}")
            return None
    else:
        return None


In [3]:
results = test()
# print(results)
print("Mã AAA - từ 2018-01-01 đến 2025-01-01")
profit = results[results > 0] 
print(f"Tổng số lần mua bán:    {len(results)}")
print(f"Tổng lãi:               {profit.sum():.2f}")
print(f"Số lần có lãi:          {len(profit)}")
print(f"Trung bình lãi:         {profit.mean():.2f}")
loss = results[results < 0]
print(f"Tổng lỗ:                {loss.sum():.2f}")
print(f"Số lần lỗ:              {len(loss)}")
print(f"Trung bình lỗ:          {loss.mean():.2f}")
print(f"Tổng lãi lỗ:            {results.sum():.2f}")


Mã AAA - từ 2018-01-01 đến 2025-01-01
Tổng số lần mua bán:    37
Tổng lãi:               14.24
Số lần có lãi:          23
Trung bình lãi:         0.62
Tổng lỗ:                -20.49
Số lần lỗ:              14
Trung bình lỗ:          -1.46
Tổng lãi lỗ:            -6.25


In [4]:
listing = Listing()


Thử với nhiều mã chứng khoán hơn

In [5]:
# results = pd.DataFrame(columns=['symbol', 'profit'])

# Test random symbols
symbols = listing.all_symbols()
symbols = symbols['symbol'] 
# symbols = symbols.sample(100).tolist()

start = '2018-01-01'
end = '2025-01-01'

total_result = pd.Series()
total_profit = pd.Series()
total_breakeven = pd.Series()
total_loss = pd.Series()

for symbol in symbols:
    print(f"\nMã {symbol}:")
    results = test(symbol, start, end)
    if results is None:
        print(f"Không có dữ liệu cho mã {symbol}.")
        continue

    total_result = pd.concat([total_result, results], ignore_index=True)

    print(f"Tổng số lần mua bán:    {len(results)}")

    profit = results[results > 0] 
    total_profit = pd.concat([total_profit, profit], ignore_index=True)
    print(f"{len(profit)} lần lãi - Tổng lãi: {profit.sum():.2f} - Trung bình lãi: {profit.mean():.2f}")
    
    breakeven = results[results == 0]
    total_breakeven = pd.concat([total_breakeven, breakeven], ignore_index=True)
    print(f"{len(breakeven)} lần hòa vốn")

    loss = results[results < 0]
    total_loss = pd.concat([total_loss, loss], ignore_index=True)
    print(f"{len(loss)} lần lỗ - Tổng lỗ: {loss.sum():.2f} - Trung bình lỗ: {loss.mean():.2f}")

    print(f"Tổng lãi lỗ:            {results.sum():.2f}")


Mã YTC:


The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


Tổng số lần mua bán:    26
18 lần lãi - Tổng lãi: 85.01 - Trung bình lãi: 4.72
0 lần hòa vốn
8 lần lỗ - Tổng lỗ: -42.19 - Trung bình lỗ: -5.27
Tổng lãi lỗ:            42.82

Mã YEG:
Tổng số lần mua bán:    32
19 lần lãi - Tổng lãi: 31.28 - Trung bình lãi: 1.65
1 lần hòa vốn
12 lần lỗ - Tổng lỗ: -89.95 - Trung bình lỗ: -7.50
Tổng lãi lỗ:            -58.67

Mã YBM:


The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


Tổng số lần mua bán:    38
21 lần lãi - Tổng lãi: 7.09 - Trung bình lãi: 0.34
1 lần hòa vốn
16 lần lỗ - Tổng lỗ: -9.51 - Trung bình lỗ: -0.59
Tổng lãi lỗ:            -2.42

Mã YBC:
Tổng số lần mua bán:    28
21 lần lãi - Tổng lãi: 61.70 - Trung bình lãi: 2.94
2 lần hòa vốn
5 lần lỗ - Tổng lỗ: -8.30 - Trung bình lỗ: -1.66
Tổng lãi lỗ:            53.40

Mã XPH:
Tổng số lần mua bán:    51
22 lần lãi - Tổng lãi: 11.00 - Trung bình lãi: 0.50
3 lần hòa vốn
26 lần lỗ - Tổng lỗ: -12.40 - Trung bình lỗ: -0.48
Tổng lãi lỗ:            -1.40

Mã XMP:
Tổng số lần mua bán:    22
12 lần lãi - Tổng lãi: 14.21 - Trung bình lãi: 1.18
3 lần hòa vốn
7 lần lỗ - Tổng lỗ: -6.63 - Trung bình lỗ: -0.95
Tổng lãi lỗ:            7.58

Mã XMD:
Tổng số lần mua bán:    22
8 lần lãi - Tổng lãi: 57.00 - Trung bình lãi: 7.12
0 lần hòa vốn
14 lần lỗ - Tổng lỗ: -14.60 - Trung bình lỗ: -1.04
Tổng lãi lỗ:            42.40

Mã XMC:
Tổng số lần mua bán:    35
16 lần lãi - Tổng lãi: 4.44 - Trung bình lãi: 0.28
4 lần hòa vốn
1

...

Tổng số lần mua bán:    0
0 lần lãi - Tổng lãi: 0.00 - Trung bình lãi: nan
0 lần hòa vốn
0 lần lỗ - Tổng lỗ: 0.00 - Trung bình lỗ: nan
Tổng lãi lỗ:            0.00

Mã BCM:
Tổng số lần mua bán:    38
19 lần lãi - Tổng lãi: 69.70 - Trung bình lãi: 3.67
2 lần hòa vốn
17 lần lỗ - Tổng lỗ: -59.05 - Trung bình lỗ: -3.47
Tổng lãi lỗ:            10.65

Mã BCG:
Tổng số lần mua bán:    41
24 lần lãi - Tổng lãi: 16.48 - Trung bình lãi: 0.69
2 lần hòa vốn
15 lần lỗ - Tổng lỗ: -18.04 - Trung bình lỗ: -1.20
Tổng lãi lỗ:            -1.56

Mã BCF:
Tổng số lần mua bán:    29
15 lần lãi - Tổng lãi: 8.13 - Trung bình lãi: 0.54
3 lần hòa vốn
11 lần lỗ - Tổng lỗ: -6.84 - Trung bình lỗ: -0.62
Tổng lãi lỗ:            1.29

Mã BCE:
Tổng số lần mua bán:    41
26 lần lãi - Tổng lãi: 9.41 - Trung bình lãi: 0.36
2 lần hòa vốn
13 lần lỗ - Tổng lỗ: -17.02 - Trung bình lỗ: -1.31
Tổng lãi lỗ:            -7.61

Mã BCC:
Tổng số lần mua bán:    44
19 lần lãi - Tổng lãi: 18.79 - Trung bình lãi: 0.99
6 lần hòa vốn
19 lần

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


Tổng số lần mua bán:    0
0 lần lãi - Tổng lãi: 0.00 - Trung bình lãi: nan
0 lần hòa vốn
0 lần lỗ - Tổng lỗ: 0.00 - Trung bình lỗ: nan
Tổng lãi lỗ:            0.00

Mã ATS:
Tổng số lần mua bán:    23
12 lần lãi - Tổng lãi: 31.50 - Trung bình lãi: 2.62
0 lần hòa vốn
11 lần lỗ - Tổng lỗ: -44.50 - Trung bình lỗ: -4.05
Tổng lãi lỗ:            -13.00

Mã ATG:
Tổng số lần mua bán:    24
7 lần lãi - Tổng lãi: 0.65 - Trung bình lãi: 0.09
1 lần hòa vốn
16 lần lỗ - Tổng lỗ: -9.24 - Trung bình lỗ: -0.58
Tổng lãi lỗ:            -8.59

Mã ATB:
Tổng số lần mua bán:    44
14 lần lãi - Tổng lãi: 3.70 - Trung bình lãi: 0.26
11 lần hòa vốn
19 lần lỗ - Tổng lỗ: -4.80 - Trung bình lỗ: -0.25
Tổng lãi lỗ:            -1.10

Mã ATA:
Tổng số lần mua bán:    30
12 lần lãi - Tổng lãi: 3.30 - Trung bình lãi: 0.27
8 lần hòa vốn
10 lần lỗ - Tổng lỗ: -3.00 - Trung bình lỗ: -0.30
Tổng lãi lỗ:            0.30

Mã AST:
Tổng số lần mua bán:    35
22 lần lãi - Tổng lãi: 54.49 - Trung bình lãi: 2.48
0 lần hòa vốn
13 lần l

The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.
The behavior of array concatenation with empty entries is deprecated. In a future version, this will no longer exclude empty items when determining the result dtype. To retain the old behavior, exclude the empty entries before the concat operation.


Tổng số lần mua bán:    0
0 lần lãi - Tổng lãi: 0.00 - Trung bình lãi: nan
0 lần hòa vốn
0 lần lỗ - Tổng lỗ: 0.00 - Trung bình lỗ: nan
Tổng lãi lỗ:            0.00

Mã AIC:
Tổng số lần mua bán:    27
15 lần lãi - Tổng lãi: 16.40 - Trung bình lãi: 1.09
2 lần hòa vốn
10 lần lỗ - Tổng lỗ: -9.90 - Trung bình lỗ: -0.99
Tổng lãi lỗ:            6.50

Mã AGX:
Tổng số lần mua bán:    33
21 lần lãi - Tổng lãi: 58.04 - Trung bình lãi: 2.76
6 lần hòa vốn
6 lần lỗ - Tổng lỗ: -18.04 - Trung bình lỗ: -3.01
Tổng lãi lỗ:            40.00

Mã AGR:
Tổng số lần mua bán:    39
27 lần lãi - Tổng lãi: 21.79 - Trung bình lãi: 0.81
0 lần hòa vốn
12 lần lỗ - Tổng lỗ: -11.55 - Trung bình lỗ: -0.96
Tổng lãi lỗ:            10.24

Mã AGP:
Tổng số lần mua bán:    35
18 lần lãi - Tổng lãi: 16.09 - Trung bình lãi: 0.89
3 lần hòa vốn
14 lần lỗ - Tổng lỗ: -14.99 - Trung bình lỗ: -1.07
Tổng lãi lỗ:            1.10

Mã AGM:
Tổng số lần mua bán:    40
20 lần lãi - Tổng lãi: 20.60 - Trung bình lãi: 1.03
4 lần hòa vốn
16 lần

In [12]:
table = pd.DataFrame(columns=['Số lần', 'Tổng tiền', 'Trung bình', 'Lớn nhất', 'Nhỏ nhất'])
table.loc['Lãi'] = [len(total_profit), total_profit.sum(), total_profit.mean(), total_profit.max(), total_profit.min()]
table.loc['Lỗ'] = [len(total_loss), total_loss.sum(), total_loss.mean(), total_loss.min(), total_loss.max()]
table.loc['Hòa vốn'] = [len(total_breakeven), 0, 0, 0, 0]
table.loc['Tổng kết'] = [len(total_result), total_result.sum(), total_result.mean(), total_result.max(), total_result.min()]

In [13]:
table

,Số lần,Tổng tiền,Trung bình,Lớn nhất,Nhỏ nhất
Lãi,28808.0,39674.00,1.377187,330.0,0.01
Lỗ,20822.0,-32286.95,-1.550617,-203.0,-0.01
Hòa vốn,3800.0,0.00,0.000000,0.0,0.00
Tổng kết,53430.0,7387.05,0.138257,330.0,-203.00
